<a href="https://colab.research.google.com/github/KimJeongmin22/disney_liveaction/blob/main/03_02_%EC%96%B8%EC%96%B4%EB%B6%84%EC%84%9D_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%8B%AC%ED%99%94_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [ ]:
# konlpy 설치하기 ## https://data1000.tistory.com/33
!pip3 install jpype1==0.7.0
!pip3 install konlpy

     |████████████████████████████████| 2.7MB 7.9MB/s 
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 8.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
# 뮬란 댓글모음
## https://drive.google.com/file/d/1TwevDZaGfdRenOCofKsCm6Q1RXYXe1I6/view?usp=sharing

rawdata_downloaded = drive.CreateFile({'id': '1TwevDZaGfdRenOCofKsCm6Q1RXYXe1I6'})
rawdata_downloaded.GetContentFile('rawdata.txt')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('rawdata.txt', sep="\t")

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,김아인(ain2****),ㅋㅋ혼자보구왔는데 세상이달라보이네 보고나면 길걷는것도 재밌고 그냥 웃음나오고 행벅함,10,4470,452,2017-03-16 12:28,관람객
1,1,resese(fleb****),말이 필요없다 진짜 실사화 제대로 해놨네 역대급임 진심 디즈니에서 일 제대로 함,10,3640,369,2017-03-16 10:13,0
2,2,언제나늘(yoon****),야수의 선홍빛 잇몸미소가 아직도 눈에 선합니다 .,10,2482,260,2017-03-16 3:48,0
3,3,최강(pipo****),시사회 보고 왔습니다 일단 역대 디즈니 실사화영화중에 역대급이라 할수 있습니다 노래...,10,2175,274,2017-03-16 0:13,0
4,4,밍밍밍(221m****),예매 망설이신 분들 .. 그냥 봐요 두번 봐요 전 다섯번 볼테니ㅎㅎㅎ헿,10,1929,306,2017-03-16 10:14,0
...,...,...,...,...,...,...,...,...
13732,13732,촘(have****),동심이 되살아나는 듯한 영화! 영상미도 최고 노래도 최고! 보는 내내 눈과귀가 즐거...,10,0,0,2017-03-18 22:45,관람객
13733,13733,성규야(1203****),진짜 실망시키지 않네요 !!! 계속 생각나게하는 아름다운 영화입니다!! ♡ 안 보신...,10,0,0,2017-03-18 22:45,0
13734,13734,벚꽃(sksu****),재미있습니다. 정말 재미있습니다,10,0,0,2017-03-18 22:45,0
13735,13735,kkmr(dtdr****),어릴때본 애니메이션 장면이 생각나고 실사화 대박.. 동화속에서 놀다감,10,0,0,2017-03-18 22:43,0


In [ ]:
원본데이터 = 원본데이터.dropna(axis=0)

In [ ]:
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,김아인(ain2****),ㅋㅋ혼자보구왔는데 세상이달라보이네 보고나면 길걷는것도 재밌고 그냥 웃음나오고 행벅함,10,4470,452,2017-03-16 12:28,관람객
1,1,resese(fleb****),말이 필요없다 진짜 실사화 제대로 해놨네 역대급임 진심 디즈니에서 일 제대로 함,10,3640,369,2017-03-16 10:13,0
2,2,언제나늘(yoon****),야수의 선홍빛 잇몸미소가 아직도 눈에 선합니다 .,10,2482,260,2017-03-16 3:48,0
3,3,최강(pipo****),시사회 보고 왔습니다 일단 역대 디즈니 실사화영화중에 역대급이라 할수 있습니다 노래...,10,2175,274,2017-03-16 0:13,0
4,4,밍밍밍(221m****),예매 망설이신 분들 .. 그냥 봐요 두번 봐요 전 다섯번 볼테니ㅎㅎㅎ헿,10,1929,306,2017-03-16 10:14,0
...,...,...,...,...,...,...,...,...
13732,13732,촘(have****),동심이 되살아나는 듯한 영화! 영상미도 최고 노래도 최고! 보는 내내 눈과귀가 즐거...,10,0,0,2017-03-18 22:45,관람객
13733,13733,성규야(1203****),진짜 실망시키지 않네요 !!! 계속 생각나게하는 아름다운 영화입니다!! ♡ 안 보신...,10,0,0,2017-03-18 22:45,0
13734,13734,벚꽃(sksu****),재미있습니다. 정말 재미있습니다,10,0,0,2017-03-18 22:45,0
13735,13735,kkmr(dtdr****),어릴때본 애니메이션 장면이 생각나고 실사화 대박.. 동화속에서 놀다감,10,0,0,2017-03-18 22:43,0


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    # 대상 텍스트는 3번이 아닌 2번입니다.
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    # score 기준으로 분류는 4번이 아닌 3번입니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

[('와', 'Verb'),
 ('진짜', 'Noun'),
 ('동심', 'Noun'),
 ('으로', 'Josa'),
 ('돌아간', 'Verb'),
 ('기분', 'Noun'),
 ('..', 'Punctuation'),
 ('결말', 'Noun'),
 ('을', 'Josa'),
 ('알고는', 'Verb'),
 ('있지만', 'Adjective'),
 ('빠져드는', 'Verb'),
 ('영화', 'Noun'),
 ('다', 'Josa'),
 ('.', 'Punctuation'),
 ('그리고', 'Conjunction'),
 ('엠마왓슨', 'Noun'),
 ('이', 'Josa'),
 ('너무', 'Adverb'),
 ('예뻐서', 'Adjective'),
 ('얼굴', 'Noun'),
 ('만', 'Josa'),
 ('봐도', 'Verb'),
 ('즐거울꺼', 'Adjective'),
 ('임', 'Noun'),
 ('진짜', 'Noun'),
 ('보는내내', 'Verb'),
 ('이쁘다는', 'Adjective'),
 ('생각', 'Noun'),
 ('밖에', 'Josa'),
 ('안', 'VerbPrefix'),
 ('들었다', 'Verb')]

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,와,Verb,10,1
1,진짜,Noun,10,1
2,동심,Noun,10,1
3,으로,Josa,10,1
4,돌아간,Verb,10,1
5,기분,Noun,10,1
6,..,Punctuation,10,1
7,결말,Noun,10,1
8,을,Josa,10,1
9,알고는,Verb,10,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[       0               1  type  count
 0     ㅋㅋ  KoreanParticle    10      1
 1     혼자            Noun    10      1
 2     보구            Noun    10      1
 3    왔는데            Verb    10      1
 4      세        Modifier    10      1
 5     상이            Noun    10      1
 6     달라            Noun    10      1
 7     보이            Noun    10      1
 8      네            Josa    10      1
 9   보고나면            Verb    10      1
 10     길            Noun    10      1
 11  걷는것도            Verb    10      1
 12   재밌고       Adjective    10      1
 13    그냥            Noun    10      1
 14    웃음            Noun    10      1
 15   나오고            Verb    10      1
 16   행벅함            Noun    10      1,        0          1  type  count
 0      말       Noun    10      1
 1      이       Josa    10      1
 2   필요없다  Adjective    10      1
 3     진짜       Noun    10      1
 4     실사       Noun    10      1
 5      화     Suffix    10      1
 6    제대로       Noun    10      1
 7    해놨네       Verb    10

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,ㅋㅋ,KoreanParticle,10,1
1,혼자,Noun,10,1
2,보구,Noun,10,1
3,왔는데,Verb,10,1
4,세,Modifier,10,1
...,...,...,...,...
27,이쁘다는,Adjective,10,1
28,생각,Noun,10,1
29,밖에,Josa,10,1
30,안,VerbPrefix,10,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,ㅋㅋ,KoreanParticle,10,1
1,혼자,Noun,10,1
2,보구,Noun,10,1
3,왔는데,Verb,10,1
4,세,Modifier,10,1
...,...,...,...,...
27,이쁘다는,Adjective,10,1
28,생각,Noun,10,1
29,밖에,Josa,10,1
30,안,VerbPrefix,10,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소    품사           분류
!      Punctuation  1      9
                    2      1
                    4      4
                    5      6
                    6     15
                          ..
힘들었지만  Adjective    10     1
힘들지    Adjective    9      1
힘듬니다   Adjective    5      1
힘듭니다   Adjective    1      1
힘차게    Adjective    10     1
Name: 카운트, Length: 33001, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소   품사          분류     
!     Punctuation 1     9
                  2     1
                  4     4
                  5     6
                  6    15
...                   ...
힘들었지만 Adjective   10    1
힘들지   Adjective   9     1
힘듬니다  Adjective   5     1
힘듭니다  Adjective   1     1
힘차게   Adjective   10    1

[33001 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')